In [86]:
# imports
# import numpy, pandas, scipy, math, matplotlib
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns


In [87]:
# estimators
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
#from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Feature selection
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold


In [88]:
# model metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score


In [89]:
# cross validation
from sklearn.model_selection import train_test_split


In [90]:
#data
LocData  = pd.read_csv("D://UnivofTexas/C5/T4/trainingData.csv")
pd.read_csv
LocData.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,...,100,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,100,100,-97,100,100,...,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807
4,100,100,100,100,100,100,100,100,100,100,...,100,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710


In [91]:
LocData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19937 entries, 0 to 19936
Columns: 529 entries, WAP001 to TIMESTAMP
dtypes: float64(2), int64(527)
memory usage: 80.5 MB


In [92]:
# check for null values in data
print(LocData.isnull().values.sum())
# There are no null values in the data 


0


In [93]:

# Create new dataframe and then convert some int variables to categorical variables
WorkData = LocData
WorkData.BUILDINGID = WorkData.BUILDINGID.astype('category')
WorkData.FLOOR = WorkData.FLOOR.astype('category')
WorkData.SPACEID = WorkData.SPACEID.astype('category')
WorkData.RELATIVEPOSITION = WorkData.RELATIVEPOSITION.astype('category')
print(WorkData['BUILDINGID'].dtypes)

category


In [94]:
LocData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19937 entries, 0 to 19936
Columns: 529 entries, WAP001 to TIMESTAMP
dtypes: category(4), float64(2), int64(523)
memory usage: 79.9 MB


In [95]:
# Drop some features 

WorkData.info()
WorkData.drop('USERID',axis=1,inplace=True)
WorkData.drop('PHONEID',axis=1,inplace=True)
WorkData.drop('TIMESTAMP',axis=1,inplace=True)
WorkData.drop('LATITUDE',axis=1,inplace=True)
WorkData.drop('LONGITUDE',axis=1,inplace=True)
WorkData.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19937 entries, 0 to 19936
Columns: 529 entries, WAP001 to TIMESTAMP
dtypes: category(4), float64(2), int64(523)
memory usage: 79.9 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19937 entries, 0 to 19936
Columns: 524 entries, WAP001 to RELATIVEPOSITION
dtypes: category(4), int64(520)
memory usage: 79.2 MB


In [96]:
WorkData.head()
# Create Subsets of data based on buildings
is_B0 =  WorkData['BUILDINGID']==0
is_B1 =  WorkData['BUILDINGID']==1
is_B2 =  WorkData['BUILDINGID']==2

WorkDataB0 = WorkData[is_B0]
WorkDataB1 = WorkData[is_B1]
WorkDataB2 = WorkData[is_B2]

print('WorkDataB0',WorkDataB0.shape)
print('WorkDataB1',WorkDataB1.shape)
print('WorkDataB2',WorkDataB2.shape)




WorkDataB0 (5249, 524)
WorkDataB1 (5196, 524)
WorkDataB2 (9492, 524)


In [97]:
WorkDataB0.info()
WorkDataB0.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5249 entries, 4 to 18497
Columns: 524 entries, WAP001 to RELATIVEPOSITION
dtypes: category(4), int64(520)
memory usage: 20.9 MB


,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION
4,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,0,0,122,2
7119,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,0,0,102,2
7120,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,0,0,110,2
7121,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,0,0,111,2
7122,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,0,0,107,2


In [98]:
# create a combined ID for location for each dataset
# Building 0 WorkDataB0
WorkDataB0['LOCATION'] = WorkDataB0["FLOOR"].astype(str) + WorkDataB0["BUILDINGID"].astype(str) + WorkDataB0["SPACEID"].astype(str) + WorkDataB0["RELATIVEPOSITION"].astype(str)
WorkDataB0.info()
print(WorkDataB0['LOCATION'].dtypes)

# Building 1 WorkDataB1
WorkDataB1['LOCATION'] = WorkDataB1["FLOOR"].astype(str) + WorkDataB1["BUILDINGID"].astype(str) + WorkDataB1["SPACEID"].astype(str) + WorkDataB1["RELATIVEPOSITION"].astype(str)
WorkDataB1.info()
print(WorkDataB1['LOCATION'].dtypes)


# Building 2 WorkDataB2
WorkDataB2['LOCATION'] = WorkDataB2["FLOOR"].astype(str) + WorkDataB2["BUILDINGID"].astype(str) + WorkDataB2["SPACEID"].astype(str) + WorkDataB2["RELATIVEPOSITION"].astype(str)
WorkDataB2.info()
print(WorkDataB2['LOCATION'].dtypes)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 5249 entries, 4 to 18497
Columns: 525 entries, WAP001 to LOCATION
dtypes: category(4), int64(520), object(1)
memory usage: 20.9+ MB
object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5196 entries, 0 to 19936
Columns: 525 entries, WAP001 to LOCATION
dtypes: category(4), int64(520), object(1)
memory usage: 20.7+ MB
object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9492 entries, 46 to 19933
Columns: 525 entries, WAP001 to LOCATION
dtypes: category(4), int64(520), object(1)
memory usage: 37.8+ MB
object


C:\Users\SANGITA\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\SANGITA\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\SANGITA\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [99]:
# set datatype for Location for each dataset
# Building 0 WorkDataB0
WorkDataB0.LOCATION = WorkDataB0.LOCATION.astype('category')
print(WorkDataB0['LOCATION'].dtypes)

# Building 1 WorkDataB1
WorkDataB1.LOCATION = WorkDataB1.LOCATION.astype('category')
print(WorkDataB1['LOCATION'].dtypes)

# Building 2 WorkDataB2
WorkDataB2.LOCATION = WorkDataB2.LOCATION.astype('category')
print(WorkDataB2['LOCATION'].dtypes)



category
category
category


In [100]:
WorkDataB0.head()


,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP516,WAP517,WAP518,WAP519,WAP520,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,LOCATION
4,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,0,0,122,2,001222
7119,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,0,0,102,2,001022
7120,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,0,0,110,2,001102
7121,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,0,0,111,2,001112
7122,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,0,0,107,2,001072


In [101]:
WorkDataB1.head()


,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP516,WAP517,WAP518,WAP519,WAP520,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,LOCATION
0,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,2,1,106,2,211062
1,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,2,1,106,2,211062
2,100,100,100,100,100,100,100,-97,100,100,...,100,100,100,100,100,2,1,103,2,211032
3,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,2,1,102,2,211022
5,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,2,1,105,2,211052


In [102]:
WorkDataB2.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP516,WAP517,WAP518,WAP519,WAP520,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,LOCATION
46,100,100,100,100,100,100,100,100,100,100,...,100,-93,100,100,100,3,2,247,2,322472
47,100,100,100,100,100,100,100,100,100,100,...,100,-89,100,100,100,3,2,248,2,322482
49,100,100,100,100,100,100,100,100,100,100,...,100,100,-91,100,100,3,2,247,1,322471
50,100,100,100,100,100,100,100,100,100,100,...,100,-90,100,100,100,3,2,246,2,322462
51,100,100,100,100,100,100,100,100,100,100,...,100,-92,100,100,100,3,2,245,2,322452


In [103]:
# # Drop individual features that were combined to create Location
#Building 0 dataset
WorkDataB0.drop('FLOOR',axis=1,inplace=True)
WorkDataB0.drop('BUILDINGID',axis=1,inplace=True)
WorkDataB0.drop('SPACEID',axis=1,inplace=True)
WorkDataB0.drop('RELATIVEPOSITION',axis=1,inplace=True)

#Building 1 dataset
WorkDataB1.drop('FLOOR',axis=1,inplace=True)
WorkDataB1.drop('BUILDINGID',axis=1,inplace=True)
WorkDataB1.drop('SPACEID',axis=1,inplace=True)
WorkDataB1.drop('RELATIVEPOSITION',axis=1,inplace=True)

#Building 2 dataset
WorkDataB2.drop('FLOOR',axis=1,inplace=True)
WorkDataB2.drop('BUILDINGID',axis=1,inplace=True)
WorkDataB2.drop('SPACEID',axis=1,inplace=True)
WorkDataB2.drop('RELATIVEPOSITION',axis=1,inplace=True)


C:\Users\SANGITA\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [104]:
WorkDataB0.info()
WorkDataB0.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5249 entries, 4 to 18497
Columns: 521 entries, WAP001 to LOCATION
dtypes: category(1), int64(520)
memory usage: 20.9 MB


,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520,LOCATION
4,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,001222
7119,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,001022
7120,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,001102
7121,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,001112
7122,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,001072


In [106]:
#features
#Building 0
featuresB0 = WorkDataB0.iloc[:,0:520]

# Building 1
featuresB1 = WorkDataB1.iloc[:,0:520]

# Building 2
featuresB2 = WorkDataB2.iloc[:,0:520]

featuresB2.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
46,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,-93,100,100,100
47,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,-89,100,100,100
49,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,-91,100,100
50,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,-90,100,100,100
51,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,-92,100,100,100


In [107]:
#dependent variable
# Building 0 dataset
depVarB0 = WorkDataB0['LOCATION']

# Building 1 dataset
depVarB1 = WorkDataB1['LOCATION']

# Building 2 dataset
depVarB2 = WorkDataB2['LOCATION']

depVarB2.head()

46    322472
47    322482
49    322471
50    322462
51    322452
Name: LOCATION, dtype: category
Categories (403, object): [021012, 021031, 021032, 021041, ..., 422172, 422182, 422192, 422202]

In [108]:
#Training Set (Feature Space: X Training) Builing 0
X_B0 = featuresB0   
y_B0 = depVarB0

from sklearn.model_selection import train_test_split
X_B0_train, X_B0_test, y_B0_train, y_B0_test = train_test_split( X_B0, y_B0, test_size=0.25, random_state = 10)

x_B0_train_count = len(X_B0_train.index)
print('The number of observations in the X training set are:',str(x_B0_train_count))
X_B0_train.head()





The number of observations in the X training set are: 3936


,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
18335,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
17224,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
18119,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
17415,100,100,100,100,100,100,100,100,-91,100,...,100,100,100,100,100,100,100,100,100,100
7309,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100


In [109]:
#Dependent Variable Training Set (y Training) Building 0
y_B0_train_count = len(y_B0_train.index)
print('The number of observations in the Y Building0 training set are:',str(y_B0_train_count))
y_B0_train.head()

The number of observations in the Y Building0 training set are: 3936


18335    102202
17224    102192
18119    201082
17415    002362
7309     301362
Name: LOCATION, dtype: category
Categories (259, object): [001022, 001062, 001072, 001102, ..., 302312, 302342, 302352, 302362]

In [110]:
# Builiding 0
#Testing Set (X Testing)
#X_test = X[-X_train:]
X_B0_test_count = len(X_B0_test.index)
print('The number of observations in the feature testing set for Building 0 is:',str(X_B0_test_count))
print(X_B0_test.head())

The number of observations in the feature testing set for Building 0 is: 1313
       WAP001  WAP002  WAP003  WAP004  WAP005  WAP006  WAP007  WAP008  WAP009  \
8957      100     100     100     100     100     100     -80     100     100   
8533      100     100     100     100     100     100     100     100     100   
17130     100     100     100     100     100     100     100     100     100   
16739     100     100     100     100     100     100     100     100     100   
18147     100     100     100     100     100     100     100     100     100   

       WAP010  ...  WAP511  WAP512  WAP513  WAP514  WAP515  WAP516  WAP517  \
8957      100  ...     100     100     100     100     100     100     100   
8533      100  ...     100     100     100     100     100     100     100   
17130     100  ...     100     100     100     100     100     100     100   
16739     100  ...     100     100     100     100     100     100     100   
18147     100  ...     100     100     100   

In [111]:
#Establish Ground truth Building 0
#Ground Truth (y_test) 
y_B0_test_count = len(y_B0_test.index)
print('The number of observations in the Y test set for Builidng 0 are:',str(y_B0_test_count))
y_B0_test.head()

The number of observations in the Y test set for Builidng 0 are: 1313


8957     102132
8533     101212
17130    002332
16739    201142
18147    201332
Name: LOCATION, dtype: category
Categories (259, object): [001022, 001062, 001072, 001102, ..., 302312, 302342, 302352, 302362]

In [112]:
# Building 0 counts
X_B0_train.shape, X_B0_test.shape


((3936, 520), (1313, 520))

In [113]:
#Models
modelRF = RandomForestClassifier(n_estimators=10)
modelgnb = GaussianNB()
modelSVM = SVC(kernel="rbf")



In [114]:
#RandomForest B0
modelRF.fit(X_B0_train,y_B0_train)
RFpredB0 = modelRF.predict( X_B0_test )
 
scoresRFB0 = cross_val_score( modelRF, X_B0_train, y_B0_train)    
print(scoresRFB0)
print("Mean RandomForest CrossVal Accuracy on Train Set %.2f, with std=%.2f" % (scoresRFB0.mean(), scoresRFB0.std() ))
print("Accuracy on test set:",metrics.accuracy_score(y_B0_test, RFpredB0))

C:\Users\SANGITA\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[0.6241033  0.62223915 0.64524003]
Mean RandomForest CrossVal Accuracy on Train Set 0.63, with std=0.01
Accuracy on test set: 0.6443259710586443


In [115]:
# Kappa RF B0
cohen_scoreRFB0 = cohen_kappa_score(y_B0_test, RFpredB0)
print("Cohen score RF B0:",cohen_scoreRFB0)

Cohen score RF B0: 0.6428238983782436


In [116]:
# kernel SVM 

modelSVM.fit( X_B0_train, y_B0_train )
SVMpredB0 = modelSVM.predict( X_B0_test )

scoresSVMB0 = cross_val_score( modelSVM, X_B0_train, y_B0_train)    
print(scoresSVMB0)
print("Mean SVM CrossVal Accuracy on Train Set %.2f, with std=%.2f" % (scoresSVMB0.mean(), scoresSVMB0.std() ))
print("Accuracy on test set:",metrics.accuracy_score(y_B0_test, SVMpredB0))


C:\Users\SANGITA\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\SANGITA\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\SANGITA\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\SANGITA\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change fro

[0.23888092 0.24600152 0.26606998]
Mean SVM CrossVal Accuracy on Train Set 0.25, with std=0.01
Accuracy on test set: 0.27722772277227725


In [117]:
# Kappa SVM B0
cohen_scoreSVMB0 = cohen_kappa_score(y_B0_test, SVMpredB0)
print("Cohen score SVM B0:",cohen_scoreSVMB0)

Cohen score SVM B0: 0.2739642936220297


In [118]:
# Naive Bayes 
modelgnb.fit( X_B0_train, y_B0_train )
GNBpredB0 = modelgnb.predict( X_B0_test )

scoresGNBB0 = cross_val_score( modelgnb, X_B0_train, y_B0_train)    
print(scoresGNBB0)
print("Mean GNB CrossVal Accuracy on Train Set %.2f, with std=%.2f" % (scoresGNBB0.mean(), scoresGNBB0.std() ))
print("Accuracy on test set:",metrics.accuracy_score(y_B0_test, GNBpredB0))


C:\Users\SANGITA\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[0.46987088 0.47143945 0.49471115]
Mean GNB CrossVal Accuracy on Train Set 0.48, with std=0.01
Accuracy on test set: 0.48667174409748665


In [119]:
# Kappa GNB B0
cohen_scoreGNBB0 = cohen_kappa_score(y_B0_test, GNBpredB0)
print("Cohen score GNB B0:",cohen_scoreGNBB0)

Cohen score GNB B0: 0.4845155811127717


In [120]:
#predictions: RF is the chosen model for predictions.
RFpredB0

array(['102012', '101202', '002332', ..., '002182', '002142', '101242'],
      dtype=object)

In [121]:
#Training Set (Feature Space: X Training) Builing 1
X_B1 = featuresB1  
y_B1 = depVarB1

from sklearn.model_selection import train_test_split
X_B1_train, X_B1_test, y_B1_train, y_B1_test = train_test_split( X_B1, y_B1, test_size=0.25, random_state = 10)

x_B1_train_count = len(X_B1_train.index)
print('The number of observations in the X training set for Building 1 are:',str(x_B1_train_count))
X_B1_train.head()


The number of observations in the X training set for Building 1 are: 3897


,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
13452,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
9616,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
6580,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
547,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
14442,100,100,100,100,100,100,100,-91,100,100,...,100,100,100,100,100,100,100,100,100,100


Random Forest is the chosen classifer. Now run the RF model for Building 1 and Building 2 datasets

In [122]:
#Dependent Variable Training Set (y Training) Building 1
y_B1_train_count = len(y_B1_train.index)
print('The number of observations in the Building 1 training set are:',str(y_B1_train_count))
y_B1_train.head()

The number of observations in the Building 1 training set are: 3897


13452    111082
9616     011082
6580     212021
547      211012
14442    111061
Name: LOCATION, dtype: category
Categories (243, object): [01101, 01102, 011072, 011082, ..., 3151, 3152, 3161, 3162]

In [123]:
# Builiding 1 Test Set
X_B1_test_count = len(X_B1_test.index)
print('The number of observations in the feature testing set for Building 1 is:',str(X_B1_test_count))
print(X_B1_test.head())

The number of observations in the feature testing set for Building 1 is: 1299
       WAP001  WAP002  WAP003  WAP004  WAP005  WAP006  WAP007  WAP008  WAP009  \
4227      100     100     100     100     100     100     100     100     100   
3396      100     100     100     100     100     100     100     100     100   
14111     100     100     100     100     100     100     100     -85     100   
5049      100     100     100     100     100     100     100     100     100   
5248      100     100     100     100     100     100     100     100     100   

       WAP010  ...  WAP511  WAP512  WAP513  WAP514  WAP515  WAP516  WAP517  \
4227      100  ...     100     100     100     100     100     100     100   
3396      100  ...     100     100     100     100     100     100     100   
14111     100  ...     100     100     100     100     100     100     100   
5049      100  ...     100     100     100     100     100     100     100   
5248      100  ...     100     100     100   

In [124]:
#Establish Ground truth Building 1
#Ground Truth (y_test) 
y_B1_test_count = len(y_B1_test.index)
print('The number of observations in the Y test set for Builidng 1 are:',str(y_B1_test_count))
y_B1_test.head()

The number of observations in the Y test set for Builidng 1 are: 1299


4227     012021
3396     012021
14111    111052
5049     311042
5248     312022
Name: LOCATION, dtype: category
Categories (243, object): [01101, 01102, 011072, 011082, ..., 3151, 3152, 3161, 3162]

In [125]:
# Building 1 counts
X_B1_train.shape, X_B1_test.shape

((3897, 520), (1299, 520))

In [127]:
# Run the chosen model RF for Building 1  
#RandomForest B1
modelRF.fit(X_B1_train,y_B1_train)

RFpredB1 = modelRF.predict( X_B1_test )
 
scoresRFB1 = cross_val_score( modelRF, X_B1_train, y_B1_train)    
print(scoresRFB1)
print("Mean RandomForest CrossVal Accuracy on Train Set for builing 1 %.2f, with std=%.2f" % (scoresRFB1.mean(), scoresRFB1.std() ))
print("Accuracy on test set for RF Building 1:",metrics.accuracy_score(y_B1_test, RFpredB1))


C:\Users\SANGITA\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\SANGITA\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[0.75593952 0.77941176 0.75740132]
Mean RandomForest CrossVal Accuracy on Train Set for builing 1 0.76, with std=0.01
Accuracy on test set for RF Building 1: 0.7990762124711316


In [128]:
# Kappa RF B1
cohen_scoreRFB1 = cohen_kappa_score(y_B1_test, RFpredB1)
print("Cohen score RF B1:",cohen_scoreRFB1)

Cohen score RF B1: 0.7977702409299352


RF on  Building 2 dataset

In [133]:
#Training Set (Feature Space: X Training) Builing 2
X_B2 = featuresB2  
y_B2 = depVarB2

from sklearn.model_selection import train_test_split
X_B2_train, X_B2_test, y_B2_train, y_B2_test = train_test_split( X_B2, y_B2, test_size=0.25, random_state = 10)

x_B2_train_count = len(X_B2_train.index)
print('The number of observations in the X training set for Building 2 are:',str(x_B2_train_count))
X_B2_train.head()


The number of observations in the X training set for Building 2 are: 7119


,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
18547,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-74,-86,100,100,100
14909,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,-50,100,100,100
19184,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-73,100,100,100,100
12226,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,-74,100,100,100
11629,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,-59,100,100,100


In [134]:
#Dependent Variable Training Set (y Training) Building 2
y_B2_train_count = len(y_B2_train.index)
print('The number of observations in the Building 2 training set are:',str(y_B2_train_count))
y_B2_train.head()

The number of observations in the Building 2 training set are: 7119


18547    021172
14909    121242
19184    021392
12226    421042
11629    221202
Name: LOCATION, dtype: category
Categories (403, object): [021012, 021031, 021032, 021041, ..., 422172, 422182, 422192, 422202]

In [135]:
# Builiding 2 Test Set
X_B2_test_count = len(X_B2_test.index)
print('The number of observations in the feature testing set for Building 2 is:',str(X_B2_test_count))
print(X_B2_test.head())

The number of observations in the feature testing set for Building 2 is: 2373
       WAP001  WAP002  WAP003  WAP004  WAP005  WAP006  WAP007  WAP008  WAP009  \
13285     100     100     100     100     100     100     100     100     100   
6621      100     100     100     100     100     100     100     100     100   
5414      100     100     100     100     100     100     100     100     100   
13042     100     100     100     100     100     100     100     100     100   
1165      100     100     100     100     100     100     100     100     100   

       WAP010  ...  WAP511  WAP512  WAP513  WAP514  WAP515  WAP516  WAP517  \
13285     100  ...     100     100     100     100     100     100     -85   
6621      100  ...     100     100     100     100     100     -79     100   
5414      100  ...     100     100     100     100     100     -54     -82   
13042     100  ...     -89     100     100     100     100     100     100   
1165      100  ...     100     100     100   

In [136]:
#Establish Ground truth Building 2
#Ground Truth (y_test) 
y_B2_test_count = len(y_B2_test.index)
print('The number of observations in the Y test set for Builidng 2 are:',str(y_B2_test_count))
y_B2_test.head()

The number of observations in the Y test set for Builidng 2 are: 2373


13285    221442
6621     021392
5414     021102
13042    321342
1165     422152
Name: LOCATION, dtype: category
Categories (403, object): [021012, 021031, 021032, 021041, ..., 422172, 422182, 422192, 422202]

In [137]:
# Building 2 counts
X_B2_train.shape, X_B2_test.shape

((7119, 520), (2373, 520))

In [138]:
# Run the chosen model RF for Building 2  
#RandomForest B2
modelRF.fit(X_B2_train,y_B2_train)

RFpredB2 = modelRF.predict( X_B2_test )
 
scoresRFB2 = cross_val_score( modelRF, X_B2_train, y_B2_train)    
print(scoresRFB2)
print("Mean RandomForest CrossVal Accuracy on Train Set for builing 1 %.2f, with std=%.2f" % (scoresRFB2.mean(), scoresRFB2.std() ))
print("Accuracy on test set for RF Building 2:",metrics.accuracy_score(y_B2_test, RFpredB2))


C:\Users\SANGITA\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\SANGITA\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[0.63647295 0.63053564 0.64580559]
Mean RandomForest CrossVal Accuracy on Train Set for builing 1 0.64, with std=0.01
Accuracy on test set for RF Building 2: 0.6894226717235566


In [139]:
# Kappa RF B2
cohen_scoreRFB2 = cohen_kappa_score(y_B2_test, RFpredB2)
print("Cohen score RF B2:",cohen_scoreRFB2)

Cohen score RF B2: 0.6884147322433112
